In [1]:
"""
from google.colab import drive
drive.mount('/content/gdrive')
"""

"\nfrom google.colab import drive\ndrive.mount('/content/gdrive')\n"

In [2]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Sequential
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [3]:
IMAGE_SIZE = [224, 224]

Now just remember the architecture in mind and start adding the layers into the network. our Kernel_size is (3,3) and the pool_size is (2,2) always.

In [4]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
model.add(Flatten(name='flatten'))
model.add(Dense(256, activation='relu', name='fc1'))
model.add(Dense(128, activation='relu', name='fc2'))
model.add(Dense(1, activation='sigmoid', name='output'))


In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

To use a pre-trained model just makes the middle layers non trainable and remove the last layer.

We are also going to do the same let’s makes the middle layers freeze.

In [6]:
for layer in vgg.layers:
  layer.trainable = False

We have already removed the output layer by include_top = False.

Let’s add our own output layer with only one node.

In [7]:
x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

Now we need to compile our model so that we can train it

In [8]:
model.compile(
  loss='categorical_crossentropy',
  #optimizer='adam',
  #optimizer='opt',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [9]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
mc = ModelCheckpoint('checkpoints',monitor='val_accuracy', mode='max', save_best_only=True)
# "/content/gdrive/My Drive/checkpoints"

Let’s generate training and validation data using the data generator

In [10]:
train_path="datasets/Buildings224/train"
test_path="datasets/Buildings224/test"

#train_path="/content/gdrive/My Drive/datasets/Buildings224/train"
#test_path="/content/gdrive/My Drive/datasets/Buildings224/test"

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   height_shift_range=0.3, 
                                   width_shift_range=0.3, 
                                   channel_shift_range=150.0, 
                                   brightness_range=(0.3, 0.9),
                                   validation_split=0.1) #validation split

test_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size = (224, 224),
    batch_size = 16,
    class_mode='categorical',
    subset='training') # set as training data

validation_set = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size = (224, 224),
    batch_size = 16,
    class_mode='categorical',
    subset='validation') # set as validation data

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            shuffle=False)





Found 2145 images belonging to 5 classes.
Found 236 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


Train the model

In [11]:
r = model.fit(
  train_set,
  validation_data = validation_set, 
  epochs = 5,
  callbacks=[mc,es]
)

Epoch 1/5
135/135 [==============================] - ETA: 0s - loss: 1.1921e-07 - accuracy: 0.8000WARNING:tensorflow:From /home/gecko/anaconda3/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoints/assets
135/135 [==============================] - 687s 5s/step - loss: 1.1921e-07 - accuracy: 0.8000 - val_loss: 1.1921e-07 - val_accuracy: 0.8000
Epoch 2/5
135/135 [==============================] - 778s 6s/step - loss: 1.1921e-07 - accuracy: 0.8000 - val_loss: 1.1921e-07 - val_accuracy: 0.8000
Epoch 3/5
135/135 [==============================] - 797s 6s/step - loss: 1

Load weights instead of training next time

In [12]:
model.load_weights("/content/gdrive/My Drive/best_model.h5")

OSError: Unable to open file (unable to open file: name = '/content/gdrive/My Drive/best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

test the model

In [13]:
model.evaluate(test_set)

16/16 [==============================] - 26s 2s/step - loss: 1.1921e-07 - accuracy: 0.8000


[1.1920928955078125e-07, 0.8000001311302185]

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

print("predicting... ")

#Confution Matrix and Classification Report
Y_pred = model.predict(test_set, len(test_set), verbose = 1)


predicting... 
16/16 [==============================] - 27s 2s/step


In [ ]:
#print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
#print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_set.labels, y_pred))
print('Classification Report')
target_names = ['Arch_Bridges', 'Castles', 'Churches', 'Towers', 'Triumphal_Arches']
print(classification_report(test_set.classes, y_pred, target_names=target_names))